In [1]:
from pandas import read_csv
import pandas as pd
from sklearn.model_selection import train_test_split
import autogluon as ag
from autogluon import TabularPrediction as task

d:\gitprojects\machinelearning\auto_ml\auto_gluon\lib\site-packages\mxnet\optimizer\optimizer.py:163: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  warnings.warn('WARNING: New optimizer %s.%s is overriding '


In [2]:
test = read_csv("NTI_data/test.csv")
train_target = read_csv("NTI_data/train_target.csv")
transactions_test = read_csv("NTI_data/transactions_test.csv")
transactions_train = read_csv("NTI_data/transactions_train.csv")

In [3]:
#Train
#Признаки, связанные с суммой покупки
agg_features=transactions_train.groupby('client_id')['amount_rur'].agg(['sum','mean','max']).reset_index()

#Количество в каждой категории
amount_of_purchase=transactions_train.groupby(['client_id','small_group'])['amount_rur'].count()
counts_train=amount_of_purchase.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values='amount_rur')
counts_train=counts_train.fillna(0)
counts_train.columns=['small_group_' + str(i) for i in counts_train.columns]

#Сумма в каждой категории
sum_of_group=transactions_train.groupby(['client_id','small_group'])['amount_rur'].sum()
sum_train=sum_of_group.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values='amount_rur')
sum_train=sum_train.fillna(0)
sum_train.columns=['small_group_sum_'+str(i) for i in sum_train.columns]

#Минимум в каждой категории
min_of_group=transactions_train.groupby(['client_id','small_group'])['amount_rur'].min()
min_train=sum_of_group.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values='amount_rur')
min_train=min_train.fillna(0)
min_train.columns=['small_group_min_'+str(i) for i in min_train.columns]

#Максимум в каждой категории
max_of_group=transactions_train.groupby(['client_id','small_group'])['amount_rur'].min()
max_train=sum_of_group.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values='amount_rur')
max_train=max_train.fillna(0)
max_train.columns=['small_group_max_'+str(i) for i in max_train.columns]

#Средняя сумма в каждой категории
avg_of_group=transactions_train.groupby(['client_id','small_group'])['amount_rur'].mean()
avg_train=avg_of_group.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values='amount_rur')
avg_train=avg_train.fillna(0)
avg_train.columns=['small_group_avg_'+str(i) for i in avg_train.columns]

#Соединим все в одну таблицу
counts_train=pd.merge(sum_train,counts_train,on='client_id')
counts_train=pd.merge(avg_train,counts_train,on='client_id')
counts_train=pd.merge(min_train,counts_train,on='client_id')
counts_train=pd.merge(max_train,counts_train,on='client_id')
train=pd.merge(train_target,agg_features,on='client_id')
train=pd.merge(train,counts_train.reset_index(),on='client_id')

In [9]:
train_data = train.head(1000)

In [10]:
import numpy as np
np.bincount(train_data['bins'])

array([239, 236, 254, 271], dtype=int64)

In [11]:
print(train.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30000 entries, 0 to 29999
Columns: 1015 entries, client_id to small_group_203
dtypes: float64(1013), int64(2)
memory usage: 232.5 MB
None


In [5]:
import time
start_time = time.time()

In [6]:
predictor = task.fit(train_data=train_data, label='bins', problem_type="multiclass", auto_stack=True, verbosity=4)

No output_directory specified. Models will be saved in: AutogluonModels/ag-20201004_155732\
Beginning AutoGluon training ...
AutoGluon will save models to AutogluonModels/ag-20201004_155732\
Train Data Rows:    1000
Train Data Columns: 1015
Preprocessing data ...
Train Data Class Count: 4
Feature Generator processed 1000 data points with 919 features
Original Features:
	int features: 1
	float features: 918
	object features: 0
Generated Features:
	int features: 0
All Features:
	int features: 1
	float features: 918
	object features: 0
Total time taken in fit_transform: 1.8269999027252197
	Data preprocessing and feature engineering runtime = 1.92s ...
AutoGluon will gauge predictive performance using evaluation metric: accuracy
To change this, specify the eval_metric argument of fit()
AutoGluon will early stop models using evaluation metric: accuracy
Saving AutogluonModels/ag-20201004_155732\learner.pkl
Saving AutogluonModels/ag-20201004_155732\utils\data\X_train.pkl
Saving AutogluonModel

[1]	train_set's multi_error: 0.355556	valid_set's multi_error: 0.56
[2]	train_set's multi_error: 0.231111	valid_set's multi_error: 0.49
[3]	train_set's multi_error: 0.186667	valid_set's multi_error: 0.44
[4]	train_set's multi_error: 0.158889	valid_set's multi_error: 0.46
[5]	train_set's multi_error: 0.122222	valid_set's multi_error: 0.5
[6]	train_set's multi_error: 0.1	valid_set's multi_error: 0.46
[7]	train_set's multi_error: 0.0866667	valid_set's multi_error: 0.47
[8]	train_set's multi_error: 0.0677778	valid_set's multi_error: 0.47
[9]	train_set's multi_error: 0.0611111	valid_set's multi_error: 0.47
[10]	train_set's multi_error: 0.0511111	valid_set's multi_error: 0.46
[11]	train_set's multi_error: 0.05	valid_set's multi_error: 0.44
[12]	train_set's multi_error: 0.0388889	valid_set's multi_error: 0.45
[13]	train_set's multi_error: 0.0344444	valid_set's multi_error: 0.44
[14]	train_set's multi_error: 0.03	valid_set's multi_error: 0.45
[15]	train_set's multi_error: 0.0244444	valid_set's

[132]	train_set's multi_error: 0	valid_set's multi_error: 0.45
[133]	train_set's multi_error: 0	valid_set's multi_error: 0.45
[134]	train_set's multi_error: 0	valid_set's multi_error: 0.45
[135]	train_set's multi_error: 0	valid_set's multi_error: 0.46
[136]	train_set's multi_error: 0	valid_set's multi_error: 0.45
[137]	train_set's multi_error: 0	valid_set's multi_error: 0.45
[138]	train_set's multi_error: 0	valid_set's multi_error: 0.46
[139]	train_set's multi_error: 0	valid_set's multi_error: 0.46
[140]	train_set's multi_error: 0	valid_set's multi_error: 0.47
[141]	train_set's multi_error: 0	valid_set's multi_error: 0.46
[142]	train_set's multi_error: 0	valid_set's multi_error: 0.45
[143]	train_set's multi_error: 0	valid_set's multi_error: 0.45
[144]	train_set's multi_error: 0	valid_set's multi_error: 0.47
[145]	train_set's multi_error: 0	valid_set's multi_error: 0.46
[146]	train_set's multi_error: 0	valid_set's multi_error: 0.45
[147]	train_set's multi_error: 0	valid_set's multi_erro

Training Gradient Boosting Model for 10000 rounds...
with the following hyperparameter settings:
{'num_threads': -1, 'objective': 'multiclass', 'num_classes': 4, 'verbose': -1, 'boosting_type': 'gbdt', 'two_round': True}


[1]	train_set's multi_error: 0.36	valid_set's multi_error: 0.6
[2]	train_set's multi_error: 0.235556	valid_set's multi_error: 0.56
[3]	train_set's multi_error: 0.182222	valid_set's multi_error: 0.58
[4]	train_set's multi_error: 0.146667	valid_set's multi_error: 0.58
[5]	train_set's multi_error: 0.13	valid_set's multi_error: 0.57
[6]	train_set's multi_error: 0.114444	valid_set's multi_error: 0.56
[7]	train_set's multi_error: 0.103333	valid_set's multi_error: 0.55
[8]	train_set's multi_error: 0.0877778	valid_set's multi_error: 0.55
[9]	train_set's multi_error: 0.0766667	valid_set's multi_error: 0.52
[10]	train_set's multi_error: 0.0744444	valid_set's multi_error: 0.51
[11]	train_set's multi_error: 0.0566667	valid_set's multi_error: 0.52
[12]	train_set's multi_error: 0.0466667	valid_set's multi_error: 0.49
[13]	train_set's multi_error: 0.04	valid_set's multi_error: 0.48
[14]	train_set's multi_error: 0.0333333	valid_set's multi_error: 0.46
[15]	train_set's multi_error: 0.0288889	valid_set'

...

Training Gradient Boosting Model for 10000 rounds...
with the following hyperparameter settings:
{'num_threads': -1, 'objective': 'multiclass', 'num_classes': 4, 'verbose': -1, 'boosting_type': 'gbdt', 'two_round': True, 'learning_rate': 0.03, 'num_leaves': 128, 'feature_fraction': 0.9, 'min_data_in_leaf': 3, 'seed_value': 0}


[1]	train_set's multi_error: 0.523333	valid_set's multi_error: 0.61
[2]	train_set's multi_error: 0.121111	valid_set's multi_error: 0.44
[3]	train_set's multi_error: 0.0222222	valid_set's multi_error: 0.43
[4]	train_set's multi_error: 0.01	valid_set's multi_error: 0.43
[5]	train_set's multi_error: 0.00444444	valid_set's multi_error: 0.4
[6]	train_set's multi_error: 0.00333333	valid_set's multi_error: 0.42
[7]	train_set's multi_error: 0.00333333	valid_set's multi_error: 0.4
[8]	train_set's multi_error: 0.00333333	valid_set's multi_error: 0.37
[9]	train_set's multi_error: 0.00222222	valid_set's multi_error: 0.37
[10]	train_set's multi_error: 0.00111111	valid_set's multi_error: 0.37
[11]	train_set's multi_error: 0.00111111	valid_set's multi_error: 0.39
[12]	train_set's multi_error: 0	valid_set's multi_error: 0.4
[13]	train_set's multi_error: 0	valid_set's multi_error: 0.39
[14]	train_set's multi_error: 0	valid_set's multi_error: 0.43
[15]	train_set's multi_error: 0	valid_set's multi_error:

[132]	train_set's multi_error: 0	valid_set's multi_error: 0.39
[133]	train_set's multi_error: 0	valid_set's multi_error: 0.39
[134]	train_set's multi_error: 0	valid_set's multi_error: 0.39
[135]	train_set's multi_error: 0	valid_set's multi_error: 0.39
[136]	train_set's multi_error: 0	valid_set's multi_error: 0.39
[137]	train_set's multi_error: 0	valid_set's multi_error: 0.39
[138]	train_set's multi_error: 0	valid_set's multi_error: 0.39
[139]	train_set's multi_error: 0	valid_set's multi_error: 0.39
[140]	train_set's multi_error: 0	valid_set's multi_error: 0.39
[141]	train_set's multi_error: 0	valid_set's multi_error: 0.39
[142]	train_set's multi_error: 0	valid_set's multi_error: 0.39
[143]	train_set's multi_error: 0	valid_set's multi_error: 0.39
[144]	train_set's multi_error: 0	valid_set's multi_error: 0.39
[145]	train_set's multi_error: 0	valid_set's multi_error: 0.39
[146]	train_set's multi_error: 0	valid_set's multi_error: 0.39
[147]	train_set's multi_error: 0	valid_set's multi_erro

Saving AutogluonModels/ag-20201004_155732\models\LightGBMClassifierCustom_STACKER_l1\utils\oof.pkl
Saving AutogluonModels/ag-20201004_155732\models\LightGBMClassifierCustom_STACKER_l1\model.pkl
	0.606	 = Validation accuracy score
	541.16s	 = Training runtime
	1.35s	 = Validation runtime
Saving AutogluonModels/ag-20201004_155732\models\trainer.pkl
Loading: AutogluonModels/ag-20201004_155732\models\RandomForestClassifierGini_STACKER_l1\utils\oof.pkl
Loading: AutogluonModels/ag-20201004_155732\models\RandomForestClassifierEntr_STACKER_l1\utils\oof.pkl
Loading: AutogluonModels/ag-20201004_155732\models\ExtraTreesClassifierGini_STACKER_l1\utils\oof.pkl
Loading: AutogluonModels/ag-20201004_155732\models\ExtraTreesClassifierEntr_STACKER_l1\utils\oof.pkl
Loading: AutogluonModels/ag-20201004_155732\models\KNeighborsClassifierUnif_STACKER_l1\utils\oof.pkl
Loading: AutogluonModels/ag-20201004_155732\models\KNeighborsClassifierDist_STACKER_l1\utils\oof.pkl
Loading: AutogluonModels/ag-20201004_1557

NameError: name 'test_data' is not defined

In [7]:
end_time = time.time()

In [8]:
print("Time spent", end_time - start_time)

Time spent 2482.0023169517517


In [9]:
test_data = train.tail(1000)
performance = predictor.evaluate(test_data)

Total time taken in transform: 0.3739752769470215
Loading: AutogluonModels/ag-20201004_155732\models\weighted_ensemble_k0_l2\model.pkl
Loading: AutogluonModels/ag-20201004_155732\models\CatboostClassifier_STACKER_l0\model.pkl
Loading: AutogluonModels/ag-20201004_155732\models\ExtraTreesClassifierEntr_STACKER_l0\model.pkl
Loading: AutogluonModels/ag-20201004_155732\models\ExtraTreesClassifierGini_STACKER_l0\model.pkl
Loading: AutogluonModels/ag-20201004_155732\models\KNeighborsClassifierDist_STACKER_l0\model.pkl
Loading: AutogluonModels/ag-20201004_155732\models\KNeighborsClassifierUnif_STACKER_l0\model.pkl
Loading: AutogluonModels/ag-20201004_155732\models\LightGBMClassifierCustom_STACKER_l0\model.pkl
Loading: AutogluonModels/ag-20201004_155732\models\LightGBMClassifier_STACKER_l0\model.pkl
Loading: AutogluonModels/ag-20201004_155732\models\NeuralNetClassifier_STACKER_l0\model.pkl
Loading: AutogluonModels/ag-20201004_155732\models\RandomForestClassifierEntr_STACKER_l0\model.pkl
Loading

Predictive performance on given dataset: accuracy = 0.598


In [ ]:
performance = predictor.evaluate(test_data)